<a href="https://colab.research.google.com/github/aminekha/AI-For-2022/blob/master/AI_for_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Introduction**

AI for 2022 is a project that aims to replace the VAR technology in Football matches. It uses ML and DL to help the referee take better decisions.

The final version of this program will take as input a video and output the same video with squares around the ball, the player who has the ball, and the player closest to him from the opponent team. The program will detect when the opponent player slides to the player who has the ball.

**Step1: Detect All the players and the ball**

In [0]:
# !mkdir -p ../root/yolo

In [5]:
%cd

/root


In [7]:
%cd
!git clone https://github.com/aminekha/AI-For-2022.git
# !mv AI-For-2022 "../root/yolo"
%cd AI-For-2022

!python setup.py build_ext --inplace
!pip install -e .
!pip install .

!mkdir bin
!wget https://pjreddie.com/media/files/yolov2.weights -P /bin

/root
Cloning into 'AI-For-2022'...
remote: Enumerating objects: 249, done.
remote: Counting objects: 100% (249/249), done.
remote: Compressing objects: 100% (203/203), done.
remote: Total 249 (delta 43), reused 242 (delta 39), pack-reused 0
Receiving objects: 100% (249/249), 67.84 MiB | 62.52 MiB/s, done.
Resolving deltas: 100% (43/43), done.
/root/AI-For-2022
Compiling darkflow/cython_utils/nms.pyx because it changed.
Compiling darkflow/cython_utils/cy_yolo2_findboxes.pyx because it changed.
Compiling darkflow/cython_utils/cy_yolo_findboxes.pyx because it changed.
[1/3] Cythonizing darkflow/cython_utils/cy_yolo2_findboxes.pyx
/usr/local/lib/python3.6/dist-packages/Cython/Compiler/Main.py:367: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /root/AI-For-2022/darkflow/cython_utils/cy_yolo2_findboxes.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
[2/3] Cythonizing darkflow/cython_utils/cy_yolo_findboxes

In [8]:
!pwd

/root/AI-For-2022


In [57]:
!python flow --model cfg/yolo.cfg --load bin/yolov2.weights --labels labels.txt --demo videofile.mp4 --gpu 1.0 --saveVideo


/root/yolo/AI-For-2022/darkflow/dark/darknet.py:54: UserWarning: ./cfg/yolov2.cfg not found, use cfg/yolo.cfg instead
  cfg_path, FLAGS.model))
Parsing cfg/yolo.cfg
Loading bin/yolov2.weights ...
Successfully identified 203934260 bytes
Finished in 0.016121387481689453s
Model has a coco model name, loading coco labels.

Building net ...
Source | Train? | Layer description                | Output size
-------+--------+----------------------------------+---------------
Instructions for updating:
Colocations handled automatically by placer.
       |        | input                            | (?, 608, 608, 3)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 608, 608, 32)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 304, 304, 32)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 304, 304, 64)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 152, 152, 64)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 152, 152, 128)
 Load  |  Yep!  | conv 1x1p0_1

**Step 2: Detect Only the players near the ball**

In [9]:
# We do this to train the model to only detect person and sports ball
# Download the Pascal VOC dataset:
!curl -O https://pjreddie.com/media/files/VOCtest_06-Nov-2007.tar
!tar xf VOCtest_06-Nov-2007.tar

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  430M  100  430M    0     0  38.1M      0  0:00:11  0:00:11 --:--:-- 41.2M


In [10]:
!wget https://pjreddie.com/media/files/yolov2-tiny-voc.weights -P bin

--2019-04-06 13:58:05--  https://pjreddie.com/media/files/yolov2-tiny-voc.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.3.39
Connecting to pjreddie.com (pjreddie.com)|128.208.3.39|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 63471556 (61M) [application/octet-stream]
Saving to: ‘bin/yolov2-tiny-voc.weights’

yolov2-tiny-voc.wei 100%[===================>]  60.53M  33.6MB/s    in 1.8s    

2019-04-06 13:58:07 (33.6 MB/s) - ‘bin/yolov2-tiny-voc.weights’ saved [63471556/63471556]



In [0]:
!mv bin/yolov2-tiny-voc.weights bin/tiny-yolo-voc.weights

In [14]:
!python flow --model cfg/tiny-yolo-voc-2c.cfg --load bin/tiny-yolo-voc.weights --train --dataset "VOCdevkit/VOC2007/JPEGImages" --annotation "VOCdevkit/VOC2007/Annotations" --gpu 0.9


Parsing ./cfg/tiny-yolo-voc.cfg
Parsing cfg/tiny-yolo-voc-2c.cfg
Loading bin/tiny-yolo-voc.weights ...
Successfully identified 63471556 bytes
Finished in 0.005838155746459961s

Building net ...
Source | Train? | Layer description                | Output size
-------+--------+----------------------------------+---------------
Instructions for updating:
Colocations handled automatically by placer.
       |        | input                            | (?, 416, 416, 3)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 416, 416, 16)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 208, 208, 16)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 208, 208, 32)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 104, 104, 32)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 104, 104, 64)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 52, 52, 64)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 52, 52, 128)
 Load  |  Yep!  | maxp 2x2p0_2      

In [15]:
!python flow --model cfg/tiny-yolo-voc-2c.cfg --load bin/tiny-yolo-voc.weights --labels labels.txt --demo videofile.mp4 --gpu 1.0 --saveVideo


Parsing ./cfg/tiny-yolo-voc.cfg
Parsing cfg/tiny-yolo-voc-2c.cfg
Loading bin/tiny-yolo-voc.weights ...
Successfully identified 63471556 bytes
Finished in 0.005728006362915039s

Building net ...
Source | Train? | Layer description                | Output size
-------+--------+----------------------------------+---------------
Instructions for updating:
Colocations handled automatically by placer.
       |        | input                            | (?, 416, 416, 3)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 416, 416, 16)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 208, 208, 16)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 208, 208, 32)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 104, 104, 32)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 104, 104, 64)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 52, 52, 64)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 52, 52, 128)
 Load  |  Yep!  | maxp 2x2p0_2      